In [1]:
import os
from argparse import Namespace
from collections import Counter
import json
import re
import string
import re
import nltk
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm.auto

In [2]:
lyric_database = pd.read_csv("/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv")
lyric_database.head(5)

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [3]:
def fix_lyrics(lyrics):
    parts = re.split(r"([\n\[\]\(\)])", lyrics)
    output = " ".join(filter(None, parts))
    output = re.sub(r"([?.,!:;])",'',output)
    output = re.sub(r"in'(?= \w|\.|,|$)","ing",output.lower())
    return output

db = lyric_database.sample(frac=0.01, random_state=10) #this database has 3 million entries
db = (
    db
    .loc[db["language"] == "en"]
    .loc[db["tag"] != "misc"]
    .loc[db["tag"] != ""]
    .loc[db["tag"] != None]
    .loc[db["lyrics"] != None]
    .loc[db["lyrics"] != ""]
    .loc[db["lyrics"] != "[Instrumental]"]
)
columns_of_interest = ['id', 'title','artist', 'lyrics', 'tag']
db = db[columns_of_interest]
db["lyrics"] = db["lyrics"].apply(fix_lyrics)
db = db.sort_values(by="id")
db.head(10)

,id,title,artist,lyrics,tag
65,57,Back That Azz Up,Juvenile,[ intro juvenile ] \n cash money records takin...,rap
98,89,N.Y. State of Mind,Nas,[ produced by dj premier ] \n \n [ intro ] \n ...,rap
182,182,Swagga Like Us,JAY-Z & T.I.,[ intro mia ] \n no one on the corner has swag...,rap
539,507,Arab Money Remix,Busta Rhymes,[ intro busta rhymes & ron browz ] \n ayo some...,rap
579,556,Hip-Hop Saved My Life,Lupe Fiasco,[ intro lupe fiasco ] \n dedicate dedicate \n ...,rap
637,605,I Can’t Go to Sleep,Wu-Tang Clan,[ verse 1 ghostface killah ] \n the technique ...,rap
643,611,The Lesson Pt. 1,The Roots,[ verse 1 black thought ] \n lyrically versati...,rap
689,666,Overdose,Omen (Producer),[ jackie long ] \n and i'ma leave on one more ...,rap
710,684,Gone,Kanye West,[ produced by kanye west ] \n \n [ intro otis ...,rap
767,731,One Time 4 Your Mind,Nas,[ produced by large professor ] \n \n [ intro ...,rap
